# GraphRNNAutomaton

### Imports

In [ ]:
import time
from tqdm import tqdm
import wandb
import torch
import torch.nn as nn
import numpy as np
from exporter import read_automatas
from wandb_utilities import get_wandb_sweep_name, create_sweep
from automatonRNN import AutomatonRNN, generate_automatas, unfold_pred
from property_validator import validate_property, get_metrics
from sklearn.utils import shuffle

ENTITY = 'verification_thesis'
PROJECT = 'AutomatonRNN'
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

In [ ]:
#If some of the parameters are changed, the sweep must be recreated
automata_property = 'unique_accepting'
number_of_states = 5
alphabet_len = 2

### Data

In [ ]:
path = f'./dataset/{number_of_states}_states/{automata_property}_property_automatas'
automatas = read_automatas(path)

In [ ]:
def get_batch_graphs(dataset, batch_size):
    dataset1, dataset2 = dataset

    shuffled_ds1, shuffled_ds2 = shuffle(dataset1, dataset2, random_state=0)

    batches1 = [shuffled_ds1[i:min(len(dataset1), i+batch_size)] for i in range(0, len(dataset1), batch_size)]
    batches2 = [shuffled_ds2[i:min(len(dataset2), i+batch_size)] for i in range(0, len(dataset2), batch_size)]
    return batches1, batches2

### Creating Model

### Obtaining data from a graph

In [ ]:
padding_symbol = 0
def add_padding_to_transitions(graphs):
    max_len = max([len(g) for g in graphs])
    new_graphs = []
    for graph in graphs:
        
        len_diff = max_len-len(graph)

        new_graph = []
        padding = np.full(len_diff, padding_symbol)
        for row in graph:
            new_graph.append(np.concatenate((row, padding)))   

        for _ in range(len_diff):
            new_graph.append(np.full(max_len, padding_symbol))

        new_graphs.append(new_graph)

    return np.array(new_graphs)

def add_padding_to_final_states(all_final_states):
    max_len = max([len(fs) for fs in all_final_states])
    padded_final_states = []
    for final_states in all_final_states:
        len_diff = max_len-len(final_states)
        padding = np.full(len_diff, padding_symbol)
        padded_final_states.append(np.concatenate((final_states, padding)))
    return np.array(padded_final_states)

def add_padding_to_graph(transitions, final_states):
    return add_padding_to_transitions(transitions), add_padding_to_final_states(final_states)

In [ ]:
def get_target_conns(graphs, node, m):
    batch_size = graphs.shape[0]

    initial_pos = max(0, node - m)
    in_conns = np.array(graphs[:,initial_pos:node,node], dtype=np.float32)
    loop_con = np.array(np.expand_dims(graphs[:,node,node],1), dtype=np.float32)
    out_conns = np.array(graphs[:,node,initial_pos:node], dtype=np.float32)
    
    padding_size = max(0, m - node)
    padding = np.zeros((batch_size,padding_size),dtype=np.float32)
    y_conns = np.concatenate((padding, in_conns, loop_con, out_conns, padding), 1)
    return torch.tensor(y_conns, dtype=torch.float32)

def get_target_is_final(final_nodes, node):
    return torch.tensor(final_nodes[:,node], dtype=torch.float32).unsqueeze(-1)

def get_target_is_end(nodes, node):
    # we sum 0 to transform bools to int
    return torch.tensor((nodes == node) + 0, dtype=torch.float32).unsqueeze(-1)

def get_nodes(graphs):
    return np.array([len(g) for g in graphs])

### Loss function

In [ ]:
def compose_loss(y_hat, y, m, weighted):
    conns_hat, final_prob_hat, end_prob_hat = unfold_pred(y_hat, m)
    conns, final_prob, end_prob = unfold_pred(y, m)
    # Convert to batch and BCE loss for conns
    conns_loss = nn.BCELoss()(conns_hat, conns) * (1 if weighted else 1)
    # BCE loss for final prob
    final_prob_loss = nn.BCELoss()(final_prob_hat, final_prob) * (1.5 if weighted else 1)
    # BCE loss for end prob
    end_prob_loss = nn.BCELoss()(end_prob_hat, end_prob) * (1 if weighted else 1)

    # Total loss us the sum of all losses
    return conns_loss + final_prob_loss + end_prob_loss

### Training Loop

In [ ]:
def train_model(model, optim, dataset, criterion, epochs, batch_size, gradient_clip, loss_type):
    dataset_len = len(dataset[0])
    for epoch in tqdm(range(epochs)):
        start_time = time.time()
        loss_val = 0
        all_transitions, all_final_states = get_batch_graphs(dataset, batch_size)
        for i, transitions in enumerate(all_transitions):
            final_states = all_final_states[i]
            bs = len(final_states)

            x = model.get_sos(bs)
            h = model.get_initial_hidden(bs)

            nodes = get_nodes(transitions)
            max_node = max(nodes)

            padded_transitions, padded_final_states = add_padding_to_graph(transitions, final_states)
            all_preds = []
            all_y = []
            for node in range(max_node):
                optim.zero_grad()

                # Get targets 
                y_conns = get_target_conns(padded_transitions, node, model.m)
                y_final = get_target_is_final(padded_final_states, node)
                y_end = get_target_is_end(nodes, node)
                y = torch.cat((y_conns, y_final, y_end), 1)
                all_y.append(y.reshape(bs,1,-1))

                # Run one iteration of the model
                pred, hidden = model(x, h)
                all_preds.append(pred.reshape(bs, 1, -1))
                # Update hidden and x values for next iteration we use teacher forcing so x is y
                h = hidden.reshape(1,bs,-1)
                x = y

            # Compute the loss function
            loss = None
            #if loss_type == 'joined':
            loss = criterion(torch.cat(all_preds, 1), torch.cat(all_y, 1))
            #else:
            #    loss = compose_loss(torch.cat(all_preds, 1), torch.cat(all_y, 1), model.m, True)
                
            loss.backward()

            if gradient_clip == 'norm':
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
            if gradient_clip == 'clip':
                torch.nn.utils.clip_grad_value_(model.parameters(), .5)
            optim.step()

            loss_val += loss.item()

        print(f"Epoch {epoch}, duration: {time.time()-start_time}s -- TRAIN: loss {loss_val/dataset_len}")
        wandb.log({'train_loss': loss_val/dataset_len})
                
    return model, loss_val/dataset_len

In [ ]:
def run_training():
    with wandb.init():
        config = wandb.config
        
        m = config['sliding_window_size']
        hidden_dim = config['hidden_dim']
        automaton_rnn = AutomatonRNN(m, hidden_dim, config['recurrent_module'], config['weight_init'], 
                                     config['dropout'], config['mlp_hidden_dim'], config['rnn_input_dim'])
        

        criterion = nn.BCELoss()

        optim = torch.optim.Adam(automaton_rnn.parameters(), lr=config['lr'])

        automaton_rnn, final_loss = train_model(automaton_rnn, optim, automatas, criterion, 
                                                epochs=config['epochs'], batch_size=config['batch_size'],
                                                gradient_clip=config['gradient_clip'], loss_type=config['loss_type'])

        number_to_generate = 1000
        bs_to_generate = 20
        generated_automatons = []
        for _ in range(0, number_to_generate, bs_to_generate):
            generated_automatons = generated_automatons + generate_automatas(automaton_rnn, 5, bs_to_generate, alphabet_len)

        results = get_metrics(automata_property, generated_automatons)

        print(f'Finished training!!! Final loss: {final_loss} --- Final results: {results}')

        wandb.log(results)

In [ ]:
parameters = {
    'epochs': {'values': [15, 20, 25]},
    'batch_size': {'values': [32, 64, 128]},
    'lr': {'values': [.00005, .0002, .001]},
    'sliding_window_size': {'values': [3, 5]},
    'hidden_dim': {'values': [128, 256, 512, 1024]},
    'gradient_clip': {'values': ['clip', 'norm', 'none']},
    'loss_type': {'values': ['separated', 'joined']},
    'weight_init': {'values': ['none', 'xavier']},
    'recurrent_module': {'values': ['GRU', 'RNN']},
    'dropout': {'values': [.0, .2, .4]},
    'mlp_hidden_dim': {'values': [256, 512]},
    'rnn_input_dim': {'values': [32, 64, 128]},
}

sweep_name = get_wandb_sweep_name(automata_property, number_of_states, alphabet_len)
#sweep_id = create_sweep(f'{sweep_name} - w/TF', parameters, 'random', PROJECT, ENTITY)

wandb.agent(sweep_id='1stl9ocz', function=run_training, entity=ENTITY, project=PROJECT, count=5)